## pipeline

In [1]:
import networkx as nx
import numpy as np
import torch

import torch.nn as nn

from statistics import mean

from func.run_pipeline_super_vox import get_outlayer_of_a_3d_shape, get_crop_by_pixel_val
from func.ultis import load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset


Using backend: pytorch


In [2]:
# load graphs
from func.ultis import load_obj

graphs = load_obj("graphs_dataset_train")


In [3]:
dataset = VoxelGraphDataset(graphs)

g = dataset[0]


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  return th.as_tensor(data, dtype=dtype)


1405
torch.Size([1405, 3])
torch.Size([5327, 1])
888
torch.Size([888, 3])
torch.Size([3534, 1])
1652
torch.Size([1652, 3])
torch.Size([6150, 1])
1491
torch.Size([1491, 3])
torch.Size([5661, 1])
1316
torch.Size([1316, 3])
torch.Size([4952, 1])
978
torch.Size([978, 3])
torch.Size([4336, 1])
1544
torch.Size([1544, 3])
torch.Size([5952, 1])
1814
torch.Size([1814, 3])
torch.Size([7458, 1])
1749
torch.Size([1749, 3])
torch.Size([7247, 1])
1541
torch.Size([1541, 3])
torch.Size([6003, 1])
1416
torch.Size([1416, 3])
torch.Size([5146, 1])
1766
torch.Size([1766, 3])
torch.Size([6972, 1])
1235
torch.Size([1235, 3])
torch.Size([4581, 1])
1654
torch.Size([1654, 3])
torch.Size([6612, 1])
1131
torch.Size([1131, 3])
torch.Size([4471, 1])
1397
torch.Size([1397, 3])
torch.Size([5481, 1])
1510
torch.Size([1510, 3])
torch.Size([5524, 1])
1253
torch.Size([1253, 3])
torch.Size([4667, 1])
1309
torch.Size([1309, 3])
torch.Size([4693, 1])
1587
torch.Size([1587, 3])
torch.Size([5967, 1])
913
torch.Size([913, 3])

# TODO probably should normalize features!!!!

In [4]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)

In [5]:
from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random

model = GCN(3, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""
from torchmetrics import F1Score

f1 = F1Score(num_classes=2, average='weighted')

epoch_loss = []
epoch_val_loss = []
epoch_accuracy = []

epoch_f1score = []
epoch_f1score_val = []

epoch_accuracy_val = []
# best_val_acc = 0
best_val_loss = 1000

for e in range(500):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    for graph_number in range(len(dataset)):
    #for graph_number in range(1):
    #for graph_number in rand_graph_numbers:
        # Forward
        model.train()
        sample_graph = dataset[graph_number]
        features = sample_graph.ndata['feat']
        labels = sample_graph.ndata['label']
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']

        # create class weights
        number_positives = torch.count_nonzero(labels)
        percentage_positives = number_positives / len(labels)
        percentage_negatives = 1 - percentage_positives

        weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
        #weights = torch.tensor([0.95, 0.05])
        #print(weights)

        CELoss = nn.CrossEntropyLoss(weight=weights)
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']
        logits = model(sample_graph, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = CELoss(logits[train_mask], labels[train_mask])



        epoch_loss.append(loss.item())

        #print(loss)
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        train_f1_score = f1(pred[train_mask], labels[train_mask])
        val_f1_score = f1(pred[val_mask], labels[val_mask])


        epoch_accuracy.append(train_acc.item())
        epoch_accuracy_val.append(val_acc.item())

        epoch_f1score.append(train_f1_score.item())
        epoch_f1score_val.append(val_f1_score.item())


        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            logits = model(sample_graph, features)
            val_loss = CELoss(logits[val_mask], labels[val_mask])
            epoch_val_loss.append(val_loss.item())
        model.train()

    if e % 5 == 0:
        print('In epoch {}, loss: {:.5f}, val loss: {:.5f}, accuracy: {:.3f}, val accuracy: {:.3f}, f1score: {:.3f}, val f1score: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_val_loss), mean(epoch_accuracy), mean(epoch_accuracy_val), mean(epoch_f1score), mean(epoch_f1score_val)))

        #if mean(epoch_accuracy_val) >= best_val_acc:
        if mean(epoch_val_loss) <= best_val_loss:
            print("new best val loss")
            torch.save(model.state_dict(), "output/graph_model.pt")
            best_val_loss = mean(epoch_val_loss)
        epoch_loss = []
        epoch_val_loss = []
        epoch_accuracy = []

        epoch_accuracy_val = []
        epoch_f1score_val = []


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 0.72064, val loss: 0.70695, accuracy: 0.530, val accuracy: 0.507, f1score: 0.625, val f1score: 0.588
new best val loss
In epoch 5, loss: 0.68388, val loss: 0.68478, accuracy: 0.699, val accuracy: 0.678, f1score: 0.748, val f1score: 0.743
new best val loss
In epoch 10, loss: 0.65818, val loss: 0.64642, accuracy: 0.690, val accuracy: 0.664, f1score: 0.756, val f1score: 0.734
new best val loss
In epoch 15, loss: 0.65033, val loss: 0.62937, accuracy: 0.687, val accuracy: 0.653, f1score: 0.759, val f1score: 0.725
new best val loss
In epoch 20, loss: 0.64364, val loss: 0.61894, accuracy: 0.702, val accuracy: 0.663, f1score: 0.763, val f1score: 0.732
new best val loss
In epoch 25, loss: 0.63938, val loss: 0.61080, accuracy: 0.718, val accuracy: 0.678, f1score: 0.767, val f1score: 0.743
new best val loss
In epoch 30, loss: 0.63775, val loss: 0.60511, accuracy: 0.731, val accuracy: 0.691, f1score: 0.772, val f1score: 0.753
new best val loss
In epoch 35, loss: 0.63710, val loss

from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random
import dgl

model = GCN(1, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""

# build one big graph
graphs_list = []
for i in range(len(dataset)):
    graphs_list.append(dataset[i])

large_g = dgl.batch(graphs_list)

epoch_loss = []
epoch_accuracy = []
for e in range(1000):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    sample_graph = large_g
    features = sample_graph.ndata['feat']
    labels = sample_graph.ndata['label']

    # create class weights
    number_positives = torch.count_nonzero(labels)
    percentage_positives = number_positives / len(labels)
    percentage_negatives = 1 - percentage_positives

    weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
    #weights = torch.tensor([0.95, 0.05])
    #print(weights)

    CELoss = nn.CrossEntropyLoss(weight=weights)
    #train_mask = sample_graph.ndata['train_mask']
    #val_mask = sample_graph.ndata['val_mask']
    logits = model(sample_graph, features)

    # Compute prediction
    pred = logits.argmax(1)

    # Compute loss
    # Note that you should only compute the losses of the nodes in the training set.
    loss = CELoss(logits, labels)
    epoch_loss.append(loss.item())
    #print(loss)
    # Compute accuracy on training/validation/test
    train_acc = (pred == labels).float().mean()
    epoch_accuracy.append(train_acc.item())
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 20 == 0:
        print('In epoch {}, loss: {:.3f}, accuracy: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_accuracy)))
        epoch_loss = []
        epoch_accuracy = []




# Train on GPU
g = g.to('cuda')
model = GCN(1, 16, dataset.num_classes).to('cuda')
train(g, model)

In [6]:
g

Graph(num_nodes=1405, num_edges=5327,
      ndata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float64)})

In [7]:

g.ndata['feat'].shape

torch.Size([1405, 3])

In [8]:
model.eval()
with torch.no_grad():
    predictions = model(g, g.ndata['feat']).argmax(1).numpy()

In [9]:
np.unique(predictions)

array([0, 1])

In [10]:
labels = g.ndata['label']
np.unique(labels)

array([0, 1])

In [11]:
len(labels[labels==1])

1315

In [12]:
len(labels[labels==0])

90

In [13]:
len(predictions[predictions==1])

1019

In [14]:
len(predictions[predictions==0])

386

In [15]:
graph_model = GCN(3, num_classes=2)
checkpoint_graph = 'output/graph_model.pt'
graph_model.load_state_dict(torch.load(checkpoint_graph))
graph_model.eval()

GCN(
  (conv1): GraphConv(in=3, out=4, normalization=both, activation=None)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): GraphConv(in=4, out=2, normalization=both, activation=None)
)

In [16]:
predictions = graph_model(g, g.ndata['feat']).argmax(1).numpy()


In [17]:

len(predictions[predictions==1])

983

In [18]:
len(predictions[predictions==0])

422